# XML形式

## データサンプル

## XML形式のデータを読み込む

In [ ]:
import xml.etree.ElementTree as ET

tree = ET.parse("data/ueno-map.kml")
root = tree.getroot()

In [ ]:
namespace = "{http://www.opengis.net/kml/2.2}"
name = root[0].find(f"{namespace}name").text
name

In [ ]:
folders = root[0].findall(f"{namespace}Folder")  # Folderノードを取得  # ・・①
out = []
for i, folder in enumerate(folders):  # Folderノードを繰り返し処理
    folder_name = folder.find(
        f"{namespace}name"
    ).text  # Folderノードのnameがカテゴリー名  # ・・②

    shops = folder.findall(f"{namespace}Placemark")  # 店舗の情報にアクセス
    shops_list = []
    for shop in shops:  # ・・③
        shop_name = shop.find(f"{namespace}name").text  # 店舗の名称を取得
        coordinates = (
            shop.find(f"{namespace}Point")
            .find(f"{namespace}coordinates")
            .text  # ・・④
        )  # 経度、緯度、高度の情報を取得
        lon, lat, alt = [
            s.strip() for s in coordinates.split(",", 2)  # ・・⑤
        ]  # 情報を分割
        # 店舗の情報を辞書にしてリストに追記
        shops_list.append(  # ・・⑥
            {
                "name": shop_name,
                "lon": lon,
                "lat": lat,
                "alt": alt,
            }
        )

    # カテゴリーごとの情報を辞書型にして、リストに追記
    out.append(  # ・・⑦
        {
            "id": i + 1,
            "name": folder_name,
            "shops": shops_list,
        }
    )
out

## pandasのオブジェクトに変換する

In [ ]:
import pandas as pd

df = pd.json_normalize(
    out,
    record_path="shops",
    meta=["name"],
    record_prefix="shop.",
)

In [ ]:
df.head()

In [ ]:
df.tail()

### Note: RSS

In [ ]:
import feedparser

url = "https://blog.python.org/feeds/posts/default?alt=rss"
d = feedparser.parse(url)

In [ ]:
d["feed"]["title"]  # RSSのタイトルを取得

In [ ]:
for entry in d.entries:
    print(entry.date, end=", ")  # エントリーの日付を出力
    print(entry.title)  # エントリーごとのタイトルを出力